In [373]:
import json, copy, re
import pandas as pd

In [374]:
with open("uca.json", 'r') as file:
    data = json.load(file)
    
profiles = copy.deepcopy(data)
articles_data = copy.deepcopy(data)

# Transform profile Data

#### Transform the `Article` and `Spciality`

In [375]:
for item in profiles:
    item["Spciality"] = ", ".join(item["Spciality"])
    item["Article"] = len(item["Article"])

profiles_df = pd.DataFrame(profiles)
# profiles_df

In [376]:
#profiles_df

#### Add 'Faculty'

In [377]:
from faculty_transform import transform_faculties
transform_faculties(profiles)

In [378]:
#pd.DataFrame(profiles)

# Transform Article Data

In [379]:
articles = [article for item in articles_data for article in item["Article"]]
pd.DataFrame(articles)

In [380]:
def decompose_description(description):
    # print(description)
    first_digit_index = next((i for i, c in enumerate(description) if c.isdigit()), None)

    article_revue = description[:first_digit_index].strip()
    remaining_info = description[first_digit_index:].strip()

    components = [item.strip() for item in remaining_info.split(',')]
    # print(components)
    volume_number = ""
    year = ""
    pages = ""
    volume = ""
    number = ""
    # Extracting values
    if len(components) == 1:
        year = components[0]
    else:
        volume_number = components[0]
        if len(components)>2:
            pages = components[1]
            year = components[2]
        else:
            pages = ""
            year = components[1]

    # Further splitting volume and number
    if volume_number != "":
        volume_number_ = [part.strip() for part in volume_number.split('(')]
        if len(volume_number_) > 1:
            volume, number = [part.strip() for part in volume_number.split('(')]
        else:
            volume = volume_number_[0]
            number = ""
    return {
        'Revue': article_revue,
        'Volume': volume,
        'Number': number.replace(")", ""),
        'Pages': pages,
        'Year': year,
    }

def decompose_articles(articles):
    articles_decomposed = []
    for article in articles:
            desc = article["description"]
            information = decompose_description(desc)
            author_data = {
                "Article_Title": article["Nom Article"],
                #"Author": author,
                "Year": article["Annee"],
                "Description": desc,
                'Revue': information['Revue'],
                'Volume': re.sub(r'\D', '', information['Volume'].replace(" …", "")),
                #'Volume': information['Volume'].replace(" …", ""),
                'Number': re.sub(r'\D', '', information['Number']),
                #'Number': information['Number'].replace(r"\w+", ""),
                'Pages': re.sub(r'\D', '', information['Pages']),
                #'Pages': information['Pages'],
            }
            articles_decomposed.append(author_data)
    return articles_decomposed

In [381]:
articles_v2 = decompose_articles(articles)
articles_v2_df = pd.DataFrame(articles_v2)
columns_to_replace = ['Number', 'Volume', 'Pages']
articles_v2_df[columns_to_replace] = articles_v2_df[columns_to_replace].fillna(0)
articles_v2

In [382]:
articles_v2_df.to_csv('generated/article_unique_v2.csv', index=False)